In [15]:
import pandas as pd

df = pd.read_csv(r'flight_data/modeled/Top_clusters20250701.csv')
df.head()

,flight_date,cluster_id,arrival_iataCode,start_min,end_min,flight_count,start_time,end_time,duration_min,midpoint_min,midpoint_time,rank,source
0,2025-07-07,0,mci,972,1132,46,16:12,18:52,160,1052,17:32,1.0,flight_90min
1,2025-07-07,1,mci,1143,1283,36,19:03,21:23,140,1213,20:13,2.0,flight_90min
2,2025-07-07,2,mci,1361,1439,30,22:41,23:59,78,1400,23:20,3.0,flight_90min
3,2025-07-08,1,mci,1320,1439,40,22:00,23:59,119,1379,22:59,1.0,flight_90min
4,2025-07-08,2,mci,1132,1283,34,18:52,21:23,151,1207,20:07,2.0,flight_90min


In [16]:
import pandas as pd
import numpy as np
import plotly.express as px


# Create 15-minute bins
bin_edges = np.arange(0, 1441, 15)
bin_labels = [f"{int(b//60):02d}:{int(b%60):02d}" for b in bin_edges[:-1]]

# Compute density per day
density_records = []

for date in df['flight_date'].unique():
    day_df = df[df['flight_date'] == date]
    for b_start, b_end, label in zip(bin_edges[:-1], bin_edges[1:], bin_labels):
        count = ((day_df['start_min'] < b_end) & (day_df['end_min'] > b_start)).sum()
        density_records.append({
            'flight_date': date,
            'time_bin': label,
            'active_clusters': count
        })

density_df = pd.DataFrame(density_records)

In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


# Create 15-minute bins
bin_edges = np.arange(0, 1441, 15)
bin_labels = [f"{int(b//60):02d}:{int(b%60):02d}" for b in bin_edges[:-1]]

# Compute density for each (date, source) pair
density_records = []
for (date, source), group in df.groupby(['flight_date', 'source']):
    for b_start, b_end, label in zip(bin_edges[:-1], bin_edges[1:], bin_labels):
        count = ((group['start_min'] < b_end) & (group['end_min'] > b_start)).sum()
        density_records.append({
            'flight_date': date,
            'source': source,
            'time_bin': label,
            'active_clusters': count
        })

density_df = pd.DataFrame(density_records)

# Create traces for each (date, source) combo
fig = go.Figure()
combinations = density_df.groupby(['flight_date', 'source'])

trace_map = []
for i, ((date, source), group) in enumerate(combinations):
    fig.add_trace(go.Bar(
        x=group['time_bin'],
        y=group['active_clusters'],
        name=f'{date} | {source}',
        visible=(i == 0)
    ))
    trace_map.append((date, source))

# Create dropdowns
date_options = sorted(density_df['flight_date'].unique())
source_options = sorted(density_df['source'].unique())

# Initial state
initial_date = date_options[0]
initial_source = source_options[0]

# Dropdown for date
date_buttons = []
for d in date_options:
    visibility = [(date == d and source == initial_source) for date, source in trace_map]
    date_buttons.append(dict(label=d, method='update', args=[{'visible': visibility}]))

# Dropdown for source
source_buttons = []
for s in source_options:
    visibility = [(date == initial_date and source == s) for date, source in trace_map]
    source_buttons.append(dict(label=s, method='update', args=[{'visible': visibility}]))

# Layout with both dropdowns
fig.update_layout(
    updatemenus=[
        dict(buttons=date_buttons, direction='down', x=0.1, xanchor='left', y=1.15, yanchor='top', showactive=True),
        dict(buttons=source_buttons, direction='down', x=0.4, xanchor='left', y=1.15, yanchor='top', showactive=True)
    ],
    title=f'Cluster Density on {initial_date} from {initial_source}',
    xaxis_title='Time (Military)',
    yaxis_title='Active Clusters',
    plot_bgcolor='white',
    yaxis=dict(gridcolor='lightgray'),
    bargap=0.05
)

fig.show()